### SWAGGER FUNCTIONS
Automatically derive the json from the schema, and use it as an example for the large language model to construct the next payload using prev conversations

In [56]:
# pip install prance
# pip install openapi-spec-validator

In [57]:
import prance

In [58]:
from prance import ResolvingParser

In [59]:
parser = ResolvingParser('https://petstore3.swagger.io/api/v3/openapi.json')

In [60]:
parser.specification["openapi"]

'3.0.2'

In [61]:
parser.specification["servers"][0]['url']

'/api/v3'

In [62]:
import json


In [63]:
from typing import Dict, Union, Optional, List

def get_api_info_by_operation_id(data: Dict[str, Dict[str, dict]], target_operation_id: str) -> Dict[str, Union[str, dict, Optional[Dict[str, dict]], List[str]]]:
    api_info = {
        "endpoint": None,
        "method": None,
        "path_params": {},
        "query_params": {},
        "body_schema": None,
        "servers": []
    }

    for path, methods in data["paths"].items():
        for method, details in methods.items():
            if "operationId" in details and details["operationId"] == target_operation_id:
                # Extract endpoint and method
                api_info["endpoint"] = path
                api_info["method"] = method.upper()

                # Extract path parameters and their schemas
                path_params = {}
                for parameter in details.get("parameters", []):
                    if parameter["in"] == "path":
                        param_name = parameter["name"]
                        param_schema = parameter.get("schema", {})
                        path_params[param_name] = param_schema
                api_info["path_params"] = path_params

                # Extract query parameters and their schemas
                query_params = {}
                for parameter in details.get("parameters", []):
                    if parameter["in"] == "query":
                        param_name = parameter["name"]
                        param_schema = parameter.get("schema", {})
                        query_params[param_name] = param_schema
                api_info["query_params"] = query_params

                # Extract request body schema
                if "requestBody" in details:
                    request_body = details["requestBody"]
                    if "content" in request_body and "application/json" in request_body["content"]:
                        api_info["body_schema"] = request_body["content"]["application/json"]["schema"]

                # Extract server URLs
                servers = data.get("servers", [])
                server_urls = [server["url"] for server in servers]
                api_info["servers"] = server_urls

                return api_info

In [64]:
get_api_info_by_operation_id(parser.specification, 'findPetsByStatus')

{'endpoint': '/pet/findByStatus',
 'method': 'GET',
 'path_params': {},
 'query_params': {'status': {'type': 'string',
   'default': 'available',
   'enum': ['available', 'pending', 'sold']}},
 'body_schema': None,
 'servers': ['/api/v3']}

In [65]:
get_api_info_by_operation_id(parser.specification, 'getPetById')

{'endpoint': '/pet/{petId}',
 'method': 'GET',
 'path_params': {'petId': {'type': 'integer', 'format': 'int64'}},
 'query_params': {},
 'body_schema': None,
 'servers': ['/api/v3']}

In [66]:
get_api_info_by_operation_id(parser.specification, 'updatePetWithForm')

{'endpoint': '/pet/{petId}',
 'method': 'POST',
 'path_params': {'petId': {'type': 'integer', 'format': 'int64'}},
 'query_params': {'name': {'type': 'string'}, 'status': {'type': 'string'}},
 'body_schema': None,
 'servers': ['/api/v3']}

In [68]:
get_api_info_by_operation_id(parser.specification, 'updateUser')

{'endpoint': '/user/{username}',
 'method': 'PUT',
 'path_params': {'username': {'type': 'string'}},
 'query_params': {},
 'body_schema': {'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64', 'example': 10},
   'username': {'type': 'string', 'example': 'theUser'},
   'firstName': {'type': 'string', 'example': 'John'},
   'lastName': {'type': 'string', 'example': 'James'},
   'email': {'type': 'string', 'example': 'john@email.com'},
   'password': {'type': 'string', 'example': '12345'},
   'phone': {'type': 'string', 'example': '12345'},
   'userStatus': {'type': 'integer',
    'description': 'User Status',
    'format': 'int32',
    'example': 1}},
  'xml': {'name': 'user'}},
 'servers': ['/api/v3']}